<a href="https://colab.research.google.com/github/nguyentuyetnhung/btap/blob/main/nhan_dang_mon_an_VN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Cài đặt thư viện cần thiết
!pip install -q gradio
!pip install -q tensorflow

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
import gradio as gr
import matplotlib.pyplot as plt

BASE_DIR = "/content/drive/MyDrive/nhandangdoan"
IMG_SIZE = (60, 60)
BATCH_SIZE = 128
EPOCHS = 300
RANDOM_STATE = 42


CANONICAL_CLASSES = ['banh chung', 'com tam', 'pho', 'banh mi']
CANONICAL_EMOJIS = ['🍚', '🍛', '🍜', '🥖']
CANONICAL_EMOJI_MAP = dict(zip(CANONICAL_CLASSES, CANONICAL_EMOJIS))

LABEL_ENCODER_PATH = os.path.join(BASE_DIR, 'label_encoder.pkl')
FINAL_MODEL_PATH = os.path.join(BASE_DIR, 'final_model.h5')


def is_image_file(filename):
    ext = filename.lower().rsplit('.', 1)
    if len(ext) == 2:
        return ext[1] in ('jpg', 'jpeg', 'png', 'bmp', 'gif', 'tiff', 'webp')
    return False

def load_data(data_dir, class_names, img_size=IMG_SIZE):
    images = []
    labels = []

    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            print(f"[Warning] Thư mục không tồn tại: {class_dir} (bỏ qua)")
            continue

        for img_name in os.listdir(class_dir):
            if not is_image_file(img_name):
                continue
            img_path = os.path.join(class_dir, img_name)
            try:
                img = Image.open(img_path).convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img, dtype=np.float32) / 255.0

                images.append(img_array)
                labels.append(class_name)
            except Exception as e:
                print(f"Lỗi đọc ảnh {img_path}: {e}")

    if len(images) == 0:
        raise ValueError(f"Không tìm thấy ảnh trong {data_dir}. Kiểm tra đường dẫn và tên thư mục lớp.")

    return np.array(images), np.array(labels)

print("Đang tải dữ liệu từ:", BASE_DIR)
try:
    X, y = load_data(BASE_DIR, CANONICAL_CLASSES, IMG_SIZE)
    print(f"Tổng ảnh: {len(X)}")

    unique, counts = np.unique(y, return_counts=True)
    print("Số lượng theo lớp:")
    for u, c in zip(unique, counts):
        print(f"  - {u}: {c} ảnh")

    missing = [c for c in CANONICAL_CLASSES if c not in unique]
    if missing:
        print(f"[Lưu ý] Không tìm thấy ảnh cho các lớp: {missing}")

except Exception as e:
    print(f"Lỗi khi tải dữ liệu: {e}")
    print("Tạo dữ liệu mẫu để demo...")

    X = np.random.rand(40, IMG_SIZE[0], IMG_SIZE[1], 3).astype(np.float32)
    y = np.array(['banh chung']*10 + ['com tam']*10 + ['pho']*10 + ['banh mi']*10)


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)

X_flat = X.reshape(X.shape[0], IMG_SIZE[0] * IMG_SIZE[1] * 3)
y_categorical = to_categorical(y_encoded, num_classes=num_classes)

# Lưu label encoder
with open(LABEL_ENCODER_PATH, 'wb') as f:
    pickle.dump({'label_encoder': label_encoder, 'trained_classes': label_encoder.classes_.tolist()}, f)
print(f"Đã lưu label encoder -> {LABEL_ENCODER_PATH}")


min_count = np.min(np.bincount(y_encoded)) if len(y_encoded) > 0 else 0
if min_count < 2:
    print('[Warning] Ít hơn 2 ảnh cho một hoặc nhiều lớp. Sẽ không dùng stratify để chia dữ liệu.')
    stratify_param = None
else:
    stratify_param = y_encoded

x_train, x_test, y_train, y_test = train_test_split(
    X_flat, y_categorical, test_size=0.2, random_state=RANDOM_STATE, stratify=stratify_param
)

print(f"Kích thước train: {x_train.shape}, test: {x_test.shape}")

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(IMG_SIZE[0]*IMG_SIZE[1]*3,)))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()
-
print("Bắt đầu huấn luyện...")
history = model.fit(
    x_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    verbose=1
)

model.save(FINAL_MODEL_PATH)
print(f"Đã lưu mô hình cuối cùng -> {FINAL_MODEL_PATH}")


print("Đang đánh giá trên tập test...")
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Độ chính xác trên tập test: {test_acc:.4f}")

try:
    with open(LABEL_ENCODER_PATH, 'rb') as f:
        le_data = pickle.load(f)
        label_encoder = le_data['label_encoder']
        trained_classes = le_data.get('trained_classes', label_encoder.classes_.tolist())
except:
    print("Không thể load label encoder, sử dụng encoder hiện tại")
    trained_classes = label_encoder.classes_.tolist()


emoji_map = {c: CANONICAL_EMOJI_MAP.get(c, '🍽️') for c in trained_classes}


def predict_image(img):
    if img is None:
        return "<p style='color: red;'>Không có ảnh được tải lên.</p>"

    try:
        img_proc = img.resize(IMG_SIZE).convert('RGB')
        arr = np.array(img_proc, dtype=np.float32) / 255.0

        arr_flat = arr.reshape(1, IMG_SIZE[0] * IMG_SIZE[1] * 3)

        preds = model.predict(arr_flat, verbose=0)
        pred_idx = int(np.argmax(preds, axis=1)[0])


        class_name = label_encoder.inverse_transform([pred_idx])[0]
        confidence = float(np.max(preds))

        # Lấy emoji (nếu có)
        emoji = emoji_map.get(class_name, '🍽️')

        # Gradient màu theo độ tin cậy
        if confidence > 0.7:
            color_gradient = "linear-gradient(135deg, #43e97b 0%, #38f9d7 100%)"
        elif confidence > 0.5:
            color_gradient = "linear-gradient(135deg, #fa709a 0%, #fee140 100%)"
        else:
            color_gradient = "linear-gradient(135deg, #ff5858 0%, #f09819 100%)"


        per_class_html = ""
        for i, cname in enumerate(label_encoder.classes_):
            cemoji = emoji_map.get(cname, '🍽️')
            per_class_html += f"<p style=\"margin:5px 0;font-size:14px;\">{cemoji} {cname}: {preds[0][i]:.2%}</p>"

        result = f"""
        <div style="text-align: center; padding: 25px; background: {color_gradient};
                    border-radius: 20px; color: white; box-shadow: 0 10px 30px rgba(0,0,0,0.2);">
            <h2 style="margin: 0; font-size: 28px; font-weight: bold;">{emoji} {class_name}</h2>
            <div style="background: rgba(255,255,255,0.2); padding: 15px; border-radius: 15px; margin: 15px 0;">
                <p style="margin: 0; font-size: 20px;">Độ tin cậy: <strong>{confidence:.2%}</strong></p>
            </div>
            <div style="background: rgba(0,0,0,0.1); padding: 10px; border-radius: 10px;">
                <p style="margin: 0; font-size: 16px;">Xác suất cho các món:</p>
                {per_class_html}
            </div>
        </div>
        """

        return result

    except Exception as e:
        return f"<p style='color:red;'>Lỗi khi dự đoán: {str(e)}</p>"


css = """
body {background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; margin: 0; padding: 20px;}
.gr-box {background: rgba(255, 255, 255, 0.95) !important; border-radius: 20px !important; padding: 25px !important;
         box-shadow: 0 15px 35px rgba(0, 0, 0, 0.2) !important; border: none !important;}
.gr-button {background: linear-gradient(135deg, #6a11cb 0%, #2575fc 100%) !important; color: white !important;
            border: none !important; border-radius: 10px !important; padding: 15px 30px !important;
            font-size: 18px !important; font-weight: bold !important; transition: all 0.3s ease !important;
            cursor: pointer !important; margin-top: 15px !important;}
.gr-button:hover {transform: translateY(-3px) !important; box-shadow: 0 8px 20px rgba(0, 0, 0, 0.3) !important;}
.footer {text-align: center; margin-top: 30px; color: rgba(255,255,255,0.7); font-size: 0.9em;}
.instructions {background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 20px; border-radius: 15px; color: white; margin-bottom: 20px;}
"""

with gr.Blocks(css=css, theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🍜 Nhận Diện Món Ăn Việt Nam
        <div style="text-align: center; color: #555; margin-bottom: 20px; font-size: 18px;">
            Ứng dụng trí tuệ nhân tạo nhận diện các món ăn Việt Nam
        </div>
        """
    )

    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            gr.Markdown("### 📸 Tải lên hình ảnh món ăn")
            image_input = gr.Image(type="pil", label="", height=300)
            predict_btn = gr.Button("🔍 Nhận diện món ăn")

            gr.Markdown("### 🍽️ Món ăn được hỗ trợ")
            for i, cls in enumerate(CANONICAL_CLASSES):
                gr.Markdown(f"{CANONICAL_EMOJIS[i]} **{cls}**")

        with gr.Column(scale=1, min_width=300):
            gr.Markdown("### 🔍 Kết quả nhận diện")
            output_html = gr.HTML()

    with gr.Row():
        with gr.Accordion("📖 Hướng dẫn sử dụng", open=False):
            gr.Markdown("""
            1. **Tải lên hình ảnh**: Click vào khung ảnh hoặc kéo thả ảnh vào
            2. **Nhận diện**: Nhấn nút "Nhận diện món ăn"
            3. **Xem kết quả**: Kết quả sẽ hiển thị ở khung bên phải

            **Lưu ý**: Ảnh càng rõ nét thì kết quả càng chính xác!
            """)

    predict_btn.click(fn=predict_image, inputs=image_input, outputs=output_html)

    gr.Markdown(
        f"""
        <div class="footer">
            Ứng dụng nhận diện món ăn Việt Nam | Sử dụng mạng ANN | Độ chính xác (test): {test_acc*100:.2f}%
        </div>
        """
    )

# Chạy giao diện
print("Giao diện nhận diện món ăn đã sẵn sàng!")
try:
    demo.launch(share=True, debug=True)
except Exception as e:
    print(f"Lỗi khi khởi chạy Gradio: {e}")
    print("Thử chạy với cờ share=False...")
    demo.launch(share=False, debug=True)

Mounted at /content/drive
Đang tải dữ liệu từ: /content/drive/MyDrive/nhandangdoan
Tổng ảnh: 84
Số lượng theo lớp:
  - banh chung: 21 ảnh
  - banh mi: 21 ảnh
  - com tam: 21 ảnh
  - pho: 21 ảnh
Đã lưu label encoder -> /content/drive/MyDrive/nhandangdoan/label_encoder.pkl
Kích thước train: (67, 10800), test: (17, 10800)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 512)            │     5,530,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 4)              │         1,028 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,662,468 (21.60 MB)

 Trainable params: 5,662,468 (21.60 MB)

 Non-trainable params: 0 (0.00 B)

Bắt đầu huấn luyện...
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1493 - loss: 1.5903 - val_accuracy: 0.2353 - val_loss: 43.7879
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.2537 - loss: 39.8987 - val_accuracy: 0.2353 - val_loss: 32.2257
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 281ms/step - accuracy: 0.2687 - loss: 29.8147 - val_accuracy: 0.2941 - val_loss: 33.5661
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.2090 - loss: 32.1412 - val_accuracy: 0.2353 - val_loss: 14.7722
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 0.2537 - loss: 13.3349 - val_accuracy: 0.2941 - val_loss: 6.1970
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.2985 - loss: 9.2807 - val_accuracy: 0.2353 - val_loss: 7.3813
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.2836 - loss: 7.3157 - val_accuracy: 0.2941 - val_loss: 3.7249
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.2537 - loss: 5.6344

Đã lưu mô hình cuối cùng -> /content/drive/MyDrive/nhandangdoan/final_model.h5
Đang đánh giá trên tập test...
1/1 - 0s - 45ms/step - accuracy: 0.5294 - loss: 1.6509
Độ chính xác trên tập test: 0.5294
Giao diện nhận diện món ăn đã sẵn sàng!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db6712386a28b46e41.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
